#  Задание
1. Создать таблицу в базе film_table_ВашаФамилия
2. Опеределить необходимые столбцы
3. Загруть в таблицу данные с сайта https://lostfilm.info/top250/
4. С помощью SQL найти лучший старый и худший новый фильм

In [3]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import pymysql

In [4]:
db_conn = pymysql.connect(host='139.162.170.240',
                         user='nordic',
                         passwd ='password0',
                         db='nordicDB')

cursor = db_conn.cursor()

In [5]:
def get_html(url: str) -> str:
    responce = requests.get(url)
    if responce.status_code == 200:
        return responce.text
    else:
        return None

In [6]:
url = 'https://lostfilm.info/top250/'

In [7]:
html = get_html(url)

if not html is None:
    soup = BeautifulSoup(html)
else:
    print('Страница не загрузилась')

In [8]:
lst_image = []
lst_name_film = []
lst_year_film = []
lst_rating = []
for i in range(0,250):
    lst_image.append("http://lostfilm.info/"+soup.find_all('div', class_ = 'cover-container')[i].find('img').get('src'))
for i in range(5,255):
    lst_name_film.append(soup.find_all('a')[i].text)
for i in range(6,1004,4):
    lst_year_film.append(soup.find_all('td')[i].text)
for i in range(7,1005,4):
    lst_rating.append(soup.find_all('td')[i].text)
data = {'image' : lst_image,
       'name_film' : lst_name_film,
       'year_film' : lst_year_film,
       'rating' : lst_rating}

In [9]:
df = pd.DataFrame(data)

In [10]:
query = """
    CREATE TABLE film_table_Balaev_K
    ( 
         Image TEXT NOT NULL, 
         Name_film TEXT NOT NULL, 
         Year TEXT NOT NULL,  
         Rating TEXT NOT NULL
         
    );
"""

cursor.execute(query)
db_conn.commit()

In [11]:
query = """
    INSERT INTO film_table_Balaev_K(
                Image, Name_film, Year, Rating
                ) VALUES
    (%s, %s, %s, %s)
"""
for row in df.iterrows():
#     print(tuple(row[1]))
# #     tpl = tuple(row[1])
    cursor.execute(query, tuple(row[1]))

In [12]:
cursor.execute("SELECT * FROM film_table_Balaev_K")

250

In [74]:
query = """
    with film_max as (
        SELECT 
            max(Year) 
        FROM film_table_Balaev_K
    )
    
    , film as (
        SELECT *
        FROM film_table_Balaev_K
            WHERE 
            film_table_Balaev_K.Year = (SELECT * FROM film_max)

    )
    , rat_min as (
        SELECT 
            min(Rating) 
        FROM film
    )
    
    SELECT * 
    FROM film
        WHERE 
            film.Rating = (SELECT * FROM rat_min)
        
    
  
"""

cursor.execute(query)
cursor.fetchall()

(('http://lostfilm.info//images/40movies/106/1053109.jpg',
  'Суд над чикагской семеркой',
  '2020',
  '7.80'),)

In [75]:
query = """
    with film_min as (
        SELECT 
            min(Year) 
        FROM film_table_Balaev_K
    )
    
    , film as (
        SELECT *
        FROM film_table_Balaev_K
            WHERE 
            film_table_Balaev_K.Year = (SELECT * FROM film_min)

    )
    , rat_max as (
        SELECT 
            max(Rating) 
        FROM film
    )
    
    SELECT * 
    FROM film
        WHERE 
            film.Rating = (SELECT * FROM rat_max)
        
    
  
"""

cursor.execute(query)
cursor.fetchall()

(('http://lostfilm.info//images/40movies/4/31304.jpg',
  'Унесенные ветром',
  '1939',
  '8.20'),)

In [76]:
db_conn.close()
cursor.close()